# Install and test AWS IoT Device SDK v2 for Python
ref: https://github.com/aws/aws-iot-device-sdk-python-v2

In [18]:
!pip install awsiotsdk

In [19]:
import argparse
from awscrt import io, mqtt, auth, http
from awsiot import mqtt_connection_builder
import sys
import threading
import time
from uuid import uuid4

In [20]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [21]:
awsiotfolder='/content/gdrive/"My Drive"/"Colab Notebooks"/CMPE181Sp2021/AWSIoT/'

In [22]:
!ls {awsiotfolder}

5d864c3651-certificate.pem.crt	5d864c3651-public.pem.key
5d864c3651-private.pem.key	AmazonRootCA1.cer


In [23]:
!cp -r {awsiotfolder} .

In [24]:
!pwd

/content


In [25]:
!ls

AWSIoT	gdrive	sample_data


In [ ]:
"test-" + str(uuid4())

'test-64c2f2b5-3f91-44dc-91e0-c17f86c20992'

In [26]:
awsiotfolder="/content/AWSIoT/"

In [34]:
class Args:
  endpoint = 'a53l6idwvzofq-ats.iot.us-west-2.amazonaws.com' #Your AWS IoT custom endpoint, not including a port
  cert = awsiotfolder+"5d864c3651-certificate.pem.crt" #File path to your client certificate, in PEM format
  key = awsiotfolder+"5d864c3651-private.pem.key" #File path to your private key, in PEM format
  rootca = awsiotfolder+"AmazonRootCA1.cer"  #File path to root certificate authority, in PEM format
  clientid = "CMPEIoT2021a" #"awsiottest-" + str(uuid4()) #Client ID for MQTT connection
  topic = "test/topic"  #"Topic to subscribe to, and publish messages to."
  message = "Hello CMPEIoT"
  count = 20 #Number of messages to publish/receive before exiting
  usewebsocket = False
  
args=Args()

In [30]:
args.key

'/content/AWSIoT/5d864c3651-private.pem.key'

In [28]:
#Use the OpenSSL s_client command to test a connection to the AWS IoT endpoint:
!openssl s_client -connect a53l6idwvzofq-ats.iot.us-west-2.amazonaws.com:8443 -CAfile '/content/AWSIoT/AmazonRootCA1.cer' -cert '/content/AWSIoT/5d864c3651-certificate.pem.crt' -key '/content/AWSIoT/5d864c3651-private.pem.key'

CONNECTED(00000005)
depth=2 C = US, O = Amazon, CN = Amazon Root CA 1
verify return:1
depth=1 C = US, O = Amazon, OU = Server CA 1B, CN = Amazon
verify return:1
depth=0 CN = *.iot.us-west-2.amazonaws.com
verify return:1
---
Certificate chain
 0 s:CN = *.iot.us-west-2.amazonaws.com
   i:C = US, O = Amazon, OU = Server CA 1B, CN = Amazon
 1 s:C = US, O = Amazon, OU = Server CA 1B, CN = Amazon
   i:C = US, O = Amazon, CN = Amazon Root CA 1
 2 s:C = US, O = Amazon, CN = Amazon Root CA 1
   i:C = US, ST = Arizona, L = Scottsdale, O = "Starfield Technologies, Inc.", CN = Starfield Services Root Certificate Authority - G2
 3 s:C = US, ST = Arizona, L = Scottsdale, O = "Starfield Technologies, Inc.", CN = Starfield Services Root Certificate Authority - G2
   i:C = US, O = "Starfield Technologies, Inc.", OU = Starfield Class 2 Certification Authority
---
Server certificate
-----BEGIN CERTIFICATE-----
MIIFoDCCBIigAwIBAgIQCOq2vBudTlMNb7Yaf5BvNzANBgkqhkiG9w0BAQsFADBG
MQswCQYDVQQGEwJVUzEPMA0GA1UECh

In [ ]:
[x.name for x in io.LogLevel]

['NoLogs', 'Fatal', 'Error', 'Warn', 'Info', 'Debug', 'Trace']

In [31]:
io.init_logging(getattr(io.LogLevel, 'Error'), 'stderr')

received_count = 0
received_all_event = threading.Event()

# Callback when connection is accidentally lost.
def on_connection_interrupted(connection, error, **kwargs):
    print("Connection interrupted. error: {}".format(error))


# Callback when an interrupted connection is re-established.
def on_connection_resumed(connection, return_code, session_present, **kwargs):
    print("Connection resumed. return_code: {} session_present: {}".format(return_code, session_present))

    if return_code == mqtt.ConnectReturnCode.ACCEPTED and not session_present:
        print("Session did not persist. Resubscribing to existing topics...")
        resubscribe_future, _ = connection.resubscribe_existing_topics()

        # Cannot synchronously wait for resubscribe result because we're on the connection's event-loop thread,
        # evaluate result with a callback instead.
        resubscribe_future.add_done_callback(on_resubscribe_complete)


def on_resubscribe_complete(resubscribe_future):
        resubscribe_results = resubscribe_future.result()
        print("Resubscribe results: {}".format(resubscribe_results))

        for topic, qos in resubscribe_results['topics']:
            if qos is None:
                sys.exit("Server rejected resubscribe to topic: {}".format(topic))


# Callback when the subscribed topic receives a message
def on_message_received(topic, payload, dup, qos, retain, **kwargs):
    print("Received message from topic '{}': {}".format(topic, payload))
    global received_count
    received_count += 1
    if received_count == args.count:
        received_all_event.set()

In [32]:
def connectAWSIoTMQTT(args):
    # Spin up resources
    event_loop_group = io.EventLoopGroup(1)
    host_resolver = io.DefaultHostResolver(event_loop_group)
    client_bootstrap = io.ClientBootstrap(event_loop_group, host_resolver)
    mqtt_connection = mqtt_connection_builder.mtls_from_path(
            endpoint=args.endpoint,
            cert_filepath=args.cert,
            pri_key_filepath=args.key,
            client_bootstrap=client_bootstrap,
            ca_filepath=args.rootca,
            on_connection_interrupted=on_connection_interrupted,
            on_connection_resumed=on_connection_resumed,
            client_id=args.clientid,
            clean_session=False,
            keep_alive_secs=6)
    
    print("Connecting to {} with client ID '{}'...".format(
        args.endpoint, args.clientid))

    connect_future = mqtt_connection.connect()

    # Future.result() waits until a result is available
    connect_future.result()
    print("Connected!")

    # Subscribe
    print("Subscribing to topic '{}'...".format(args.topic))
    subscribe_future, packet_id = mqtt_connection.subscribe(
        topic=args.topic,
        qos=mqtt.QoS.AT_LEAST_ONCE,
        callback=on_message_received)

    subscribe_result = subscribe_future.result()
    print("Subscribed with {}".format(str(subscribe_result['qos'])))

    # Publish message to server desired number of times.
    # This step is skipped if message is blank.
    # This step loops forever if count was set to 0.
    if args.message:
        if args.count == 0:
            print ("Sending messages until program killed")
        else:
            print ("Sending {} message(s)".format(args.count))

        publish_count = 1
        while (publish_count <= args.count) or (args.count == 0):
            message = "{} [{}]".format(args.message, publish_count)
            print("Publishing message to topic '{}': {}".format(args.topic, message))
            mqtt_connection.publish(
                topic=args.topic,
                payload=message,
                qos=mqtt.QoS.AT_LEAST_ONCE) #mqtt.QoS.AT_LEAST_ONCE, AT_MOST_ONCE
            time.sleep(1)
            publish_count += 1

    # Wait for all messages to be received.
    # This waits forever if count was set to 0.
    if args.count != 0 and not received_all_event.is_set():
        print("Waiting for all messages to be received...")

    received_all_event.wait()
    print("{} message(s) received.".format(received_count))

    # Disconnect
    print("Disconnecting...")
    disconnect_future = mqtt_connection.disconnect()
    disconnect_future.result()
    print("Disconnected!")


In [35]:
connectAWSIoTMQTT(args)

Connecting to a53l6idwvzofq-ats.iot.us-west-2.amazonaws.com with client ID 'CMPEIoT2021a'...
Connected!
Subscribing to topic 'test/topic'...
Subscribed with QoS.AT_LEAST_ONCE
Sending 20 message(s)
Publishing message to topic 'test/topic': Hello CMPEIoT [1]
Received message from topic 'test/topic': b'Hello CMPEIoT [1]'
Publishing message to topic 'test/topic': Hello CMPEIoT [2]
Received message from topic 'test/topic': b'Hello CMPEIoT [2]'
Publishing message to topic 'test/topic': Hello CMPEIoT [3]
Received message from topic 'test/topic': b'Hello CMPEIoT [3]'
Publishing message to topic 'test/topic': Hello CMPEIoT [4]
Received message from topic 'test/topic': b'Hello CMPEIoT [4]'
Publishing message to topic 'test/topic': Hello CMPEIoT [5]
Received message from topic 'test/topic': b'Hello CMPEIoT [5]'
Publishing message to topic 'test/topic': Hello CMPEIoT [6]
Received message from topic 'test/topic': b'Hello CMPEIoT [6]'
Publishing message to topic 'test/topic': Hello CMPEIoT [7]
Recei